In [1]:
import pandas as pd
import pandas as pd
import re
from bs4 import BeautifulSoup
import nltk
from hazm import sent_tokenize , word_tokenize ,Lemmatizer,Stemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import time

In [12]:
df = pd.read_excel('دربند پراسس.xlsx')
df

,Unnamed: 0,num,dates,likes,comments,url,hastags,caption
0,0,1,2022-01-23 15:45:02,7,2,CZE_1CJF1yr,"['دروازه_دولت', 'سنگلج', 'باسیون', 'منیریه', '...",#دروازه_دولت#سنگلج#باسیون\n#منیریه#عولاجان#حما...
1,1,2,2022-01-22 08:52:00,36,2,CZBqIwLBI1A,"['کوه', 'کوهنورد', 'کوهپیمایی', 'کوهنوردی', 'ک...",#کوه #کوهنورد #کوهپیمایی #کوهنوردی #کوهستان #ت...
2,2,3,2021-11-28 21:42:44,181,0,CW1b-p9j9K9,"['طبیعت', 'دریا', 'بادی', 'پاورلیفتینگ', 'باشگ...",#طبیعت#دریا #بادی #پاورلیفتینگ #باشگاه_بانوان ...
3,7,8,2022-01-29 11:27:28,122,27,CZT_A8-B-ku,"['همنورد', 'همنورد', 'همنورد', 'همنورد', 'همنو...",《#همنورد》\n#همنورد عجب امنيت خوبي ست!\nميتواني...
4,8,9,2022-01-29 11:18:07,3,0,CZT-DYXqp9N,"['کوهنوردی', 'کوه', 'برف', 'دربند', 'شیرپلا']",بار،سنگین است و من کم طاقت و دنیا حسود\n♣️\nخم...
...,...,...,...,...,...,...,...,...
2526,12744,12746,2021-09-01 09:49:55,362,12,CTRkuJ_DR3c,"['عکاسی_فضای_باز', 'عکاسی_پرتره', 'عکاسی_هنری'...",@mahdiye.ff 📸\n\n#عکاسی_فضای_باز #عکاسی_پرتره ...
2527,12746,12748,2021-09-01 09:00:19,430,8,CTRfC01j9ql,"['پرتره', 'عکاسی', 'دربند', 'چهره', 'تهران', '...",@mahdiye.ff 📸❤\n\n#پرتره #عکاسی #دربند #چهره #...
2528,12747,12749,2021-09-01 08:32:09,152,2,CTRb0fUsewd,"['کجابریم', 'رستورانگردي', 'رستورانهای_تهران',...",.\nرُست. \n.\nپ. نون: عکس برا یه گوشهٔ دننننج ...
2529,12759,12761,2021-09-01 07:03:39,37,1,CTRRrGOlFlm,"['دیزی', 'آبگوشت', 'ظهر', 'غذا', 'دربند', 'درک...",امیدوارم امروز ظهر\nدلتون پر از شادی باشه\nوخو...


In [13]:
def clean(x):
    
    #removing all html tags and removing link addresses
    try:
            cleaned_data=re.sub(r'\S*https?:\S*', ' ', BeautifulSoup(x, 'html.parser').text)

            #lower casing all eng characters
            cleaned_data=cleaned_data.lower()

            #removing extra spaces before and after sentence
            cleaned_data=cleaned_data.strip()                  

            # removing taged names
            cleaned_data=re.sub(r'@(\w+)', '', cleaned_data )
            #removing hashstags
            cleaned_data=re.sub(r'#', ' ', cleaned_data )


            # ?white space  extra dots \n
            cleaned_data=re.sub(r'\n', ' ', cleaned_data )

            cleaned_data = re.sub(r"[\*\"“”\n\\…\+\-\/\=\(\)‘•:\[\]\|’;]", " ",cleaned_data)
            cleaned_data = re.sub(r"[ ]+", " ", cleaned_data)
            cleaned_data = re.sub(r"\!+", "", cleaned_data)
            cleaned_data = re.sub(r"\,+", "", cleaned_data)
            cleaned_data = re.sub(r"\?+", "", cleaned_data)
            cleaned_data = re.sub(r"\؟+", "", cleaned_data)

            cleaned_data = re.sub(r'[_"\-;%()|.,+&=*%.,!?:#@\[\]]', ' ', cleaned_data)

            #removing time   

            cleaned_data=re.sub(r'\d{1,2}(?:(?:am|pm)|(?::\d{1,2})(?:am|pm)?)', '', cleaned_data )

            # numbers
            cleaned_data = re.sub(r"[0-9]", " ", cleaned_data)
            #dots
            cleaned_data = re.sub(r"\.", " ", cleaned_data)
            #Eng strings
            cleaned_data = re.sub(r"[a-zA-Z]+", "", cleaned_data)

            cleaned_data = re.sub(r"\d+", "", cleaned_data)
            return cleaned_data

    except:
        print (x)
        
    
                                     

df["caption2"]=df["caption"].apply(lambda x: clean(x))
df

nan


,Unnamed: 0,num,dates,likes,comments,url,hastags,caption,caption2
0,0,1,2022-01-23 15:45:02,7,2,CZE_1CJF1yr,"['دروازه_دولت', 'سنگلج', 'باسیون', 'منیریه', '...",#دروازه_دولت#سنگلج#باسیون\n#منیریه#عولاجان#حما...,دروازه دولت سنگلج باسیون منیریه عولاجان حمام ...
1,1,2,2022-01-22 08:52:00,36,2,CZBqIwLBI1A,"['کوه', 'کوهنورد', 'کوهپیمایی', 'کوهنوردی', 'ک...",#کوه #کوهنورد #کوهپیمایی #کوهنوردی #کوهستان #ت...,کوه کوهنورد کوهپیمایی کوهنوردی کوهستان توچال ...
2,2,3,2021-11-28 21:42:44,181,0,CW1b-p9j9K9,"['طبیعت', 'دریا', 'بادی', 'پاورلیفتینگ', 'باشگ...",#طبیعت#دریا #بادی #پاورلیفتینگ #باشگاه_بانوان ...,طبیعت دریا بادی پاورلیفتینگ باشگاه بانوان بیل...
3,7,8,2022-01-29 11:27:28,122,27,CZT_A8-B-ku,"['همنورد', 'همنورد', 'همنورد', 'همنورد', 'همنو...",《#همنورد》\n#همنورد عجب امنيت خوبي ست!\nميتواني...,《 همنورد》 همنورد عجب امنيت خوبي ست ميتواني با ...
4,8,9,2022-01-29 11:18:07,3,0,CZT-DYXqp9N,"['کوهنوردی', 'کوه', 'برف', 'دربند', 'شیرپلا']",بار،سنگین است و من کم طاقت و دنیا حسود\n♣️\nخم...,بار،سنگین است و من کم طاقت و دنیا حسود ♣️ خم ش...
...,...,...,...,...,...,...,...,...,...
2526,12744,12746,2021-09-01 09:49:55,362,12,CTRkuJ_DR3c,"['عکاسی_فضای_باز', 'عکاسی_پرتره', 'عکاسی_هنری'...",@mahdiye.ff 📸\n\n#عکاسی_فضای_باز #عکاسی_پرتره ...,📸 عکاسی فضای باز عکاسی پرتره عکاسی هنری دربن...
2527,12746,12748,2021-09-01 09:00:19,430,8,CTRfC01j9ql,"['پرتره', 'عکاسی', 'دربند', 'چهره', 'تهران', '...",@mahdiye.ff 📸❤\n\n#پرتره #عکاسی #دربند #چهره #...,📸❤ پرتره عکاسی دربند چهره تهران عکاسی فضای ب...
2528,12747,12749,2021-09-01 08:32:09,152,2,CTRb0fUsewd,"['کجابریم', 'رستورانگردي', 'رستورانهای_تهران',...",.\nرُست. \n.\nپ. نون: عکس برا یه گوشهٔ دننننج ...,رُست پ نون عکس برا یه گوشهٔ دننننج رستور...
2529,12759,12761,2021-09-01 07:03:39,37,1,CTRRrGOlFlm,"['دیزی', 'آبگوشت', 'ظهر', 'غذا', 'دربند', 'درک...",امیدوارم امروز ظهر\nدلتون پر از شادی باشه\nوخو...,امیدوارم امروز ظهر دلتون پر از شادی باشه وخونه...


In [14]:
with open('stopwords.txt') as stopwords_file:
    stopwords = stopwords_file.readlines()
stopwords = [line.replace('\n', '') for line in stopwords]
# nltk.download('stopwords')
# nltk_stopwords = nltk.corpus.stopwords.words('english')
# stopwords.extend(nltk_stopwords)

In [15]:
dataset = pd.DataFrame(columns=['caption2'])
for index, row in df.iterrows():
    try:
        ads_text = row['caption2'] 
        ads_text_tokenized = word_tokenize(ads_text)
        ads_text_tokenized_filtered = [w for w in ads_text_tokenized if not w in stopwords]
#         ads_text_tokenized_filtered = [Lemmatizer.lemmatize(w) for w in ads_text_tokenized_filtered]
    except:
        print(index)
    dataset.loc[index] = {
        'caption2': ' '.join(ads_text_tokenized_filtered)}

392


In [16]:
tex = df['caption2'][10]
ads_text_tokenized = word_tokenize(tex)

In [17]:
tex

'حادثه در ارتفاعات شیرپلا🔴 با ما همراه باشید 👇🔴 🔴 به گزارش دکتر ابراهیم قاسمی مسئول عملیات نجات کوهستان هلال احمر شهرستان شمیرانات استان تهران در گفتگوی تلفنی گفت متاسفانه علی رغم اعلام تمام هشدارهای هواشناسی باز هم شاهد حادثه ای در منطقه شیرپلا بودیم که با حضور نیروهای بی ادعا تیم امداد کوهستان هلال احمر این شهرستان توانستیم خانمی که در روز چهارشنبه دچار آسیب دیدگی از ناحیه زانو شده و در منطقه زمین گیر بود را در شرایط بارش برف و وزش بادی با سرعت بسیار بالا، این مصدوم را به منطقه امن و سپس به بیمارستان منتقل کنیم  خداروشکر عزیزان هلال احمر عملیاتی را مجدد با موفقیت انجام دادند و جا دارد دست این بزرگواران را ببوسیم و خسته نباشی خدماتشان عرض کنیم با تشکر از انجمن پزشکی کوهستان ایران آرزوی سلامتی برای تک تک همنوردان برفراز سرفراز ♥️🙏♥️ عشق کوه کوهنورد حادثه شیرپلا تهران توچال مصدومیت قله پر برف قله عشق دربند امداد نجات هلال احنر زمین گیر عملیات'

In [9]:
from wordcloud_fa import WordCloudFa



In [18]:
text = dataset['caption2'].to_list()
str1 = ''.join(str(e) for e in text)
str1

"دروازه دولت سنگلج باسیون منیریه عولاجان حمام چال شاهپور چارسو گذر وزیر دفتر توپخانه ناصرخسرو سرچشمه بهارستان سپهسالار خانه قدیمی ٱباد طهرون پامنار گلوبندک درب باغ ملی تجریش دربند درکهکوه کوهنورد کوهپیمایی کوهنوردی کوهستان توچال تهران توچال توچال اسپرت دربند تهران دربند شیرپلا توچال شیرپلا دربند شیرپلاطبیعت دریا بادی پاورلیفتینگ باشگاه بانوان بیلدینگ کلاسیک اندام زیبا تهران آبشار ورزش شمال بیرجند دربند پایتخت دوستانه عشق《 همنورد》 همنورد امنيت خوبي ست ميتواني خودِ باشي ميتواني دردهايت هرچندناچيز گران بي خجالت ميان بگذاري حماقت هايت همنورد انتخاب آزاد توست اختيار توست نامش شناسنامه ات نمي نويسند نامت شناسنامه نمي نويسند همنورد عرف نيست عادت نيست معذوريت نيست همنورد نسبتي مبراست همنورد سايبان دلچسبي ست خستگي ات فراموشي بسپاري 🔜®️🔚 طبیعت\xa0 طبیعتگردی طبیعت گردی کوه\xa0 کوهنوردی کوهستان کلکچال داراباد چینکلاغ توچال دربند جمشیدیه نیاوران پناهگاه سرکچال دماوند ورزشی تتلو ابی معین جک طنزسنگین طاقت دنیا حسود ♣️ خم عار میدانم دعا بشکنم 🎱🖤 کوهنوردی کوه \u200c برف دربند شیرپلا🥀 🇮🇷 برف بازی❄⛄ زمست

In [25]:
str1 = str1.replace('پالادیوم پالادیوم', 'پالادیوم')
str1 = str1.replace('کریسمس کریسمس', 'کریسمس')

str1

"جهان خلق خلق جهان زیبایی ببخشی کارت را\xa0به نحو\xa0احسن دنیا لبخند می\u200cزنند زن مبارک😍😍😍🎊🎊🎊 ❤❤❤❤❤❤❤❤❤❤ دوستای گلم خانمهای مهربون پیجم روزتون هزاران مبارک 😍😍🎊🎊🎊🎊🎊🎊🎊🎊❤❤❤❤🌺🌺🌺🌺👌👌👌👌👌 ☆☆☆☆☆☆☆☆☆☆☆☆☆ روززن روزمادر روزمادرمبارک لبخند زیبای خدا زن مادر عشق زندگی خدا ایران تهران پل طبیعت تهرانگردی بام تهران توچال باملند توچال طبیعت مادرم کلیپ مادر انگیزشی دیالوگ ماندگارتکرار میشیم تکراری🤙❤ چهره عشق غمگین چیتگر دوست پل طبیعتگرگ هیچوقت گله نمیزنه👁 پل طبیعت آبوآتش خفن عکسپروفایل عکس عکس هنری عکسپروفایل دخترونه عکس دخترافتتاحیه پل طبیعت مهر روزی پل افتتاح خاطراتی دوران ساختش تک تک بچه\u200cهای دیبا کرده_بود پل نتیجه زحمت خیلی\u200cهاست حس تهرانخاطره پل طبیعتپاتریک مغزت\u200cپر\u200cاز\u200cماسه\u200cاس باب\u200cاسفنجی مغز\u200cپر\u200cاز\u200cماسه\u200cبهتر\u200c از\u200c\u200cدل\u200cپر\u200cاز\u200cسنگه سلام شما😻🍃 عالی دلبرجان دخترا هرچقدر شیم تهه ته ته قلبمون عاشق رنگ صورتیم🙈 همش دلمون خودمون میگیم عه صورتی بزرگ\u200c مسخرت میکننا دختر😒 چشممون دنبال لباس صورتیه بستنی صورتیه دفترصورتیه ماگ صور

In [34]:
str1 = str1.replace('پالادیوم پالادیوم', 'پالادیوم')
str1 = str1.replace('کریسمس کریسمس', 'کریسمس')
str1 = str1.replace('عکاسی عکاسی' , 'عکاسی')
str1 = str1.replace('دنیا دنیاجهانبخت', 'دنیا جهانبخت')
str1 = str1.replace('پل پل','پل')
str1 = str1.replace('تهران تهرانگردی', 'تهرانگردی')
str1 = str1.replace('تهرانگردی تهران', 'تهرانگردی')
str1 = str1.replace('طبیعت طبیعت گردی', 'طبیعت گردی')
str1 = str1.replace('طبیعت طبیعت', 'طبیعت')
str1 = str1.replace('رفیق رفیق', 'رفیق')
str1 = str1.replace('اکسپلور اکسپلور', 'اکسپلور')
str1 = str1.replace('خانواده خانواده', 'خانواده')
str1 = str1.replace('دربند شیرپلا', 'شیرپلا دربند')
str1 = str1.replace('کوه کوه','کوه')
str1 = str1.replace('کوهنوردی کوه', 'کوهنوردی')
str1 = str1.replace('کوه کوهنوردی', 'کوهنوردی')
str1 = str1.replace('دربند تهران','تهران دربند')
str1 = str1.replace('دربند دربند','دربند')
str1 = str1.replace('دربندتوچال','توچال دربند')


str1

"دروازه دولت سنگلج باسیون منیریه عولاجان حمام چال شاهپور چارسو گذر وزیر دفتر توپخانه ناصرخسرو سرچشمه بهارستان سپهسالار خانه قدیمی ٱباد طهرون پامنار گلوبندک درب باغ ملی تجریش دربند درکهکوهنورد کوهپیمایی کوهنوردیستان توچال تهران توچال توچال اسپرت تهران شیرپلا دربند توچال شیرپلا شیرپلا دربندطبیعت دریا بادی پاورلیفتینگ باشگاه بانوان بیلدینگ کلاسیک اندام زیبا تهران آبشار ورزش شمال بیرجند دربند پایتخت دوستانه عشق《 همنورد》 همنورد امنيت خوبي ست ميتواني خودِ باشي ميتواني دردهايت هرچندناچيز گران بي خجالت ميان بگذاري حماقت هايت همنورد انتخاب آزاد توست اختيار توست نامش شناسنامه ات نمي نويسند نامت شناسنامه نمي نويسند همنورد عرف نيست عادت نيست معذوريت نيست همنورد نسبتي مبراست همنورد سايبان دلچسبي ست خستگي ات فراموشي بسپاري 🔜®️🔚 طبیعت\xa0 طبیعتگردی طبیعت گردی کوه\xa0 کوهنوردیستان کلکچال داراباد چینکلاغ توچال دربند جمشیدیه نیاوران پناهگاه سرکچال دماوند ورزشی تتلو ابی معین جک طنزسنگین طاقت دنیا حسود ♣️ خم عار میدانم دعا بشکنم 🎱🖤 کوهنوردی \u200c برف شیرپلا دربند🥀 🇮🇷 برف بازی❄⛄ زمستان ☃️❄ ایران تهران شمی

In [21]:
str1 = str1.replace('عکاسی عکاسی' , 'عکاسی')
str1 = str1.replace('تیر تیر' , 'تیر ')
str1 = str1.replace('تهران تهرانگردی', 'تهرانگردی')


In [35]:
wordcloud = WordCloudFa(font_path=None,
    width=1080,
    height=720,
    margin=2,
    ranks_only=None,
    prefer_horizontal=0.9,
    mask=None,
    scale=1,
    color_func=None,
    max_words=500,
    min_font_size=6,
    stopwords=None,
    random_state=None,
    background_color='white',
    max_font_size=None,
    font_step=1,
    mode='RGB',
    relative_scaling='auto',
    regexp=None,
    collocations=True,
    colormap=None,
    normalize_plurals=True,
    contour_width=0,
    contour_color='white',
    repeat=False,
    include_numbers=True)
wc = wordcloud.generate(str1)
image = wc.to_image()
image.show()
image.save('دربند.png')